<a href="https://colab.research.google.com/github/yanisamrouche/data_science/blob/main/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intégration de données TP1 : Wrapper modulaire
AMROUCHE Yanis <br> OULD-CHIBANI Abdessettar
## Description de Jeu de Données:
Les fichiers utilisés décrivent le nombre de personnes ayant reçu au moins une
dose ou complètement vaccinées.

Les données issues du système d’information Vaccin Covid permettent de
dénombrer en temps quasi réel (J-1), le nombre de personnes ayant reçu une
injection de vaccin anti-covid en tenant compte du nombre de doses reçues, du
vaccin, et du niveau géographique (national, régional et départemental). Les
données sont disponibles par lieu de vaccination.

Les 3 fichiers Utilisés sont :
* vacsi-tot-v-dep-2022-01-12-19h06.csv
* vacsi-tot-v-reg-2022-01-12-19h06.csv
* vacsi-tot-v-fra-2022-01-12-19h06.csv

## Description du scénario applicatif:
### Propagation de vaccin:
Une association médicale souhaite évaluer la situation de l’épidémie au
département de Bouches-du-Rhône. En premier il évalue le nombre total de
vaccinés dans toute la France, après elle compare la distribution de ce nombre de
vaccinés en nombre de doses dans chaque région, puis on compare la distribution
de nombre total de vaccinés dans les différents départements de la région
Provence-Alpes-Côte d'Azur

In [ ]:
#méthode pour parser un fichier CSV ===> list (colonne)
import pandas as pd
import numpy as np
def read_csv(filename):
    df = pd.read_csv(filename, sep=";")
    columns = df.columns
    columns = columns.to_numpy()
    df.loc[0] = columns
    array = df.to_numpy()
    print(array)
    return array
read_csv("vacsi-tot-v-dep-2022-01-12-19h06.csv")
    

In [ ]:
def create_table(tablename, fields):
  #creation
  sqlQuery = "CREATE TABLE IF NOT EXISTS {}".format(tablename)
  sqlQuery += " ("
  for i in fields[0]:
    sqlQuery += "'{}', ".format(i)
  sqlQuery = sqlQuery[:-2] + ");"
  #insertion
  for x in fields[1:len(fields)]:
    values = "("
    for data in x:
      values += "'{}', ".format(data)
    values = values[:-2]
    values += ")"  
    sqlQuery += "\nINSERT INTO {} VALUES {};".format(tablename, values)
  return sqlQuery
fields = read_csv("vacsi-tot-v-dep-2022-01-12-19h06.csv")
create_table("table", fields)

In [ ]:
def create_wrapper(tablename, filename, output):
  data = create_table(tablename, read_csv(filename))
  file_ = open(output, "w")
  file_.write(data)
  file_.close()
  print("{}".format(output))
create_wrapper("vaccin", "vacsi-tot-v-dep-2022-01-12-19h06.csv", "vaccinDEP")
create_wrapper("vaccin", "vacsi-tot-v-reg-2022-01-12-19h06.csv", "vaccinREG")
create_wrapper("vaccin", "vacsi-tot-v-fra-2022-01-12-19h06.csv", "vaccinFRA")

# Les vues
0 : Tous vaccins

1 : Pfizer/BioNTech

2 : Moderna

3 : AstraZeneka

4 : Janssen

# creation de la vues : les departements avec total vaccin pfizer:
```sql
DROP VIEW IF EXISTS vaccinPfizer ;
CREATE VIEW vaccinPfizer AS
SELECT VFR.vaccin, VDEP.dep, VDEP.n_tot_dose1, VDEP.n_tot_dose2, VDEP.n_tot_dose3, VDEP.n_tot_rappel 
FROM vaccinFR VFR, vaccinDEP VDEP
WHERE VFR.vaccin = VDEP.vaccin AND VDEP.vaccin = 1;
```
# creation de la vues : les departements avec total vaccin Moderna
```sql
DROP VIEW IF EXISTS vaccinModerna ;
CREATE VIEW vaccinModerna AS
SELECT VFR.vaccin, VDEP.dep, VDEP.n_tot_dose1, VDEP.n_tot_dose2, VDEP.n_tot_dose3, VDEP.n_tot_rappel 
FROM vaccinFR VFR, vaccinDEP VDEP
WHERE VFR.vaccin = VDEP.vaccin AND VDEP.vaccin = 2;
```
# creation de la vues : les departements avec total vaccin AstraZeneka
```sql
DROP VIEW IF EXISTS vaccinAstraZ ;
CREATE VIEW vaccinAstraZ AS
SELECT VFR.vaccin, VDEP.dep, VDEP.n_tot_dose1, VDEP.n_tot_dose2, VDEP.n_tot_dose3, VDEP.n_tot_rappel 
FROM vaccinFR VFR, vaccinDEP VDEP
WHERE VFR.vaccin = VDEP.vaccin AND VDEP.vaccin = 3;
```
# creation de la vues : total vaccins dans la région 93 : Provence-Alpes-Côte d’Azur 
```sql
DROP VIEW IF EXISTS vaccinREG93 ;
CREATE VIEW vaccinREG93 AS
SELECT VFR.vaccin, VREG.reg, VREG.n_tot_dose1, VREG.n_tot_dose2, VREG.n_tot_dose3, VREG.n_tot_rappel 
FROM vaccinFR VFR, vaccinREG VREG
WHERE VFR.vaccin = VREG.vaccin AND VREG.reg = 93;
```
# creation de la vues : total vaccins dans le département 13 : Bouches-du-Rhône
```sql
CREATE VIEW vaccinDEP13 AS
SELECT VFR.vaccin, VDEP.dep, VDEP.n_tot_dose1, VDEP.n_tot_dose2, VDEP.n_tot_dose3, VDEP.n_tot_rappel 
FROM vaccinFR VFR, vaccinDEP VDEP
WHERE VFR.vaccin = VDEP.vaccin AND VDEP.dep = 13;
```